In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv')


In [ ]:
pd.options.display.max_columns = 25
#show the first 5 rows
print(df.head(n=5))

In [ ]:
import matplotlib.pyplot as plt
for column in df.columns:
    plt.hist(df[column])
    plt.title(column)
    plt.xticks(rotation=30)
    plt.show()

In [ ]:
testScores=df.columns[5:8]
for count,score in enumerate(testScores):
    plt.subplot(1,3,count+1)
    plt.boxplot(df[score])
    plt.title(score)
    plt.subplots_adjust(right =1.9)
    print("\n The 25, 50 and 75 percentiles are as follows: ")
    print(np.percentile(df[score],[25,50,75]))

In [ ]:
for score in testScores:
    name =score + " pass"
    df[name]=(df[score]>(np.percentile(df[score],[25]))[0])*1

df['passed all']=df["math score pass"]+df["reading score pass"]+df["writing score pass"]

df.loc[(df['passed all'] != 3),'passed all']='failed'
df.loc[(df['passed all'] == 3),'passed all']='passed'

In [ ]:
df_forplot =df.loc[:,('passed all', 'gender','race/ethnicity','parental level of education','lunch','test preparation course')]
g = sns.catplot("passed all", col="gender", data=df_forplot,kind="count", height=2.5, aspect=.8, col_wrap=2)
r = sns.catplot("passed all", col="race/ethnicity", data=df_forplot,kind="count", height=2.5, aspect=.8, col_wrap=5)
p = sns.catplot("passed all", col="parental level of education", data=df_forplot,kind="count", height=2.5, aspect=1.4, col_wrap=6)
l = sns.catplot("passed all", col="lunch", data=df_forplot,kind="count", height=2.5, aspect=.8, col_wrap=2)
t = sns.catplot("passed all", col="test preparation course", data=df_forplot,kind="count", height=2.5, aspect=1.2, col_wrap=2)

In [ ]:
X = pd.DataFrame(df, columns=(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course']), index=df.index)
    
Y = df['passed all']

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
y = le.transform(Y)
x = pd.DataFrame(X,columns=[X.columns[0]])
for colname in X.columns:
    le.fit(X[colname])
    x[colname] = le.transform(X[colname])
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
from sklearn.naive_bayes import GaussianNB
skgnb = GaussianNB()
skgnb.fit(x_train,y_train)
train_predict = skgnb.predict(x_train)
from sklearn import metrics
print("Training accuracy:",metrics.accuracy_score(y_train, train_predict))

test_predict = skgnb.predict(x_test)
print("Testing accuracy:",metrics.accuracy_score(y_test, test_predict))